In [7]:
import numpy as np
import pandas as pd
import csv

import os, os.path
import sys

# to move files from one directory to another
import shutil

In [8]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
plots_dir = "/Users/hn/Documents/01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/"

In [9]:
# choices_set_2_xl = pd.ExcelFile(perry_dir + "set_2_handPicked.xlsx")
# choices_set_2_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names
# choices_set_2_sheet_names

In [10]:
chosen_fields = pd.read_csv(perry_dir+"set2_hand_picked_extended.csv")

print('There are [{ques_count}] questions.'.format(ques_count=chosen_fields.shape[0]))

There are [322] questions.


In [11]:
chosen_fields.columns

Index(['ID', 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', 'latitude',
       'longitude', 'Question Text', 'CropTyp', 'Irrigtn', 'DataSrc', 'Acres',
       'ExctAcr', 'LstSrvD', 'county'],
      dtype='object')

In [12]:
bad_cols = ['corrected_RGB', 'TOA_RGB', 'Question Text']
chosen_fields.drop(labels=bad_cols, axis=1, inplace=True)

In [13]:
response_set_xl = pd.ExcelFile(perry_dir + "Perry_and_Co_Responses.xlsx")
response_set_sheet_names = response_set_xl.sheet_names  # see all sheet names
response_set_sheet_names = [x for x in response_set_sheet_names if 'Set 2' in x]
response_set_sheet_names = sorted(response_set_sheet_names)
response_set_sheet_names

['Set 2-Form 1',
 'Set 2-Form 2',
 'Set 2-Form 3',
 'Set 2-Form 4',
 'Set 2-Form 5',
 'Set 2-Form 6',
 'Set 2-Form 7']

# Read all the responses

Read and concatenate all the responses. Add the ID to each row as well.

In [14]:
# sample_response = response_set_xl.parse(response_set_sheet_names[3])
# sample_response = sample_response.drop(columns=['Timestamp'])
# sample_response
# no_questions = (sample_response.shape[1]-1) // 3
# no_questions

In [15]:
question_count = chosen_fields.shape[0]
question_count

322

In [16]:
question_count = chosen_fields.shape[0]

In [17]:
response_cols = ["ID", "Set", "Form", "Question",
                "PerryV", "AndrewV", # "TimV", "KirtiV",
                "PerryD", "AndrewD", # "TimD", "KirtiD",
                "PerryC", "AndrewC", # "TimC", "KirtiC"
                ]

all_responases = pd.DataFrame(columns=response_cols, 
                             index=range(question_count))


row_number = -1
for response_sheet_name in response_set_sheet_names:
    sample_response = response_set_xl.parse(response_sheet_name)
    sample_response = sample_response.drop(columns=['Timestamp'])
    number_of_questions = (sample_response.shape[1]-1)//3
    
    for question_number in range(1, number_of_questions+1):
        # list(curr_tbl.columns)[1].split("(")[1].split(")")[0]
        row_number+=1
        
        # Pick columns corresponding to current question!
        col_start = (question_number*3)-2
        curr_tbl = sample_response.iloc[:, [0, col_start, col_start+1, col_start+2]].copy()

        #  enter the Set number
        ID = curr_tbl.columns[1].split("(")[1].split(")")[0]
        all_responases.loc[row_number, "ID"]=ID
        all_responases.loc[row_number, "Set"]=response_sheet_name.split("-")[0].split(" ")[1]
        all_responases.loc[row_number, "Form"]=response_sheet_name.split("-")[1].split(" ")[1]
        all_responases.loc[row_number, "Question"]=curr_tbl.columns[1].split(" ")[1]

        for email in sample_response["Email Address"].values:

            if "andrew" in email:
                # Andrew
                all_responases.loc[row_number, "AndrewV"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][1]
                all_responases.loc[row_number, "AndrewD"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][2]
                all_responases.loc[row_number, "AndrewC"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][3]
            elif "beale" in email:
                # Perry
                all_responases.loc[row_number, "PerryV"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][1]
                all_responases.loc[row_number, "PerryD"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][2]
                all_responases.loc[row_number, "PerryC"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][3]
            elif "kirti" in email:
                # Kirti
                all_responases.loc[row_number, "KirtiV"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][1]
                all_responases.loc[row_number, "KirtiD"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][2]
                all_responases.loc[row_number, "KirtiC"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][3]
            elif "water" in email:
                # Tim
                all_responases.loc[row_number, "TimV"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][1]
                all_responases.loc[row_number, "TimD"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][2]
                all_responases.loc[row_number, "TimC"]=curr_tbl[curr_tbl["Email Address"]==email].values[0][3]
                
all_responases.head(2)

,ID,Set,Form,Question,PerryV,AndrewV,PerryD,AndrewD,PerryC,AndrewC
0,148920_WSDA_SF_2015,2,1,1,Single Crop,Single Crop,NaN,NaN,NaN,NaN
1,152722_WSDA_SF_2015,2,1,2,Single Crop,Single Crop,NaN,NaN,NaN,NaN


In [18]:
out_name = perry_dir + "set2_all_responses.csv"
all_responases.to_csv(out_name, index = False)

In [19]:
# Drop rows that are entirely NA
# all_responases.dropna(how='all', axis=0, inplace=True)

# Drop columns that are entirely NA
# all_responases.dropna(how='all', axis=1, inplace=True)
# print (all_responases.shape)

In [20]:
# all_responases.sort_values(by=['Form', 'ID'], inplace=True)

In [21]:
all_responases = pd.merge(all_responases, chosen_fields, on=['ID'], how='left')
all_responases = all_responases.fillna("No")
all_responases.head(2)

,ID,Set,Form,Question,PerryV,AndrewV,PerryD,AndrewD,PerryC,AndrewC,NDVI_TS_Name,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,148920_WSDA_SF_2015,2,1,1,Single Crop,Single Crop,No,No,No,No,Walla2015_148920_WSDA_SF_2015.png,46.036098,-118.720122,alfalfa seed,sprinkler,wsda,2,2.197327,2015/06/04 00:00:00,Walla Walla
1,152722_WSDA_SF_2015,2,1,2,Single Crop,Single Crop,No,No,No,No,Walla2015_152722_WSDA_SF_2015.png,46.030083,-118.691968,alfalfa seed,sprinkler,wsda,33,33.308379,2015/06/04 00:00:00,Walla Walla


# Clean up the all_responases
Keep only fields that there is a disagreement, or discussion is requested, or there is a comment.
In other word, toss the ones that are good.

In [22]:
all_responases.head(2)

,ID,Set,Form,Question,PerryV,AndrewV,PerryD,AndrewD,PerryC,AndrewC,NDVI_TS_Name,latitude,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county
0,148920_WSDA_SF_2015,2,1,1,Single Crop,Single Crop,No,No,No,No,Walla2015_148920_WSDA_SF_2015.png,46.036098,-118.720122,alfalfa seed,sprinkler,wsda,2,2.197327,2015/06/04 00:00:00,Walla Walla
1,152722_WSDA_SF_2015,2,1,2,Single Crop,Single Crop,No,No,No,No,Walla2015_152722_WSDA_SF_2015.png,46.030083,-118.691968,alfalfa seed,sprinkler,wsda,33,33.308379,2015/06/04 00:00:00,Walla Walla


### Create a long PDF of fields where discussion is requested

In [ ]:
import subprocess
os.getcwd()

In [23]:
os.chdir('/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/set_2_PDFs/')

disagreement_count = 0

with open("set_2_3in1_Discussion_Comments_Disagreements.tex","w") as file:
    file.write("\\documentclass{article}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")
    file.write("\\definecolor{mgreen}{RGB}{25,147,100}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")
    

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

    file.write("\\title{\\bf Set 2 - Discussion, Comments, and Disgreements}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")


    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    # file.write("\\section{Discussion, Comments, and Disgreements}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in all_responases.index:
        curr_row = all_responases.loc[a_row]
        
        if len(np.unique(curr_row[["PerryV", "AndrewV"]].values))>1:
            is_there_disagreement=True
            disagreement_count+=1
        else:
            is_there_disagreement=False
        
        if str(curr_row["PerryD"])==str(curr_row["AndrewD"])=='No':
            discuss_or_not=False
        else:
            discuss_or_not=True
        
        if str(curr_row["PerryC"])==str(curr_row["AndrewC"])=='No':
            comment_or_not=False
        else:
            comment_or_not=True
        
        if is_there_disagreement or discuss_or_not or comment_or_not:
            TS_file_name = curr_row["NDVI_TS_Name"]
            file.write("\\begin{figure*}[ht]\n")
            file.write("\\centering\n")
            file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                         "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                             TS_file_name + "}\n")
            file.write("\\caption[]{" + \
                       "\\textbf{\\color{red}{" + curr_row["CropTyp"] + "}}" + \
                       ", ID: " + curr_row["ID"].replace("_", "\_") + \
                       ", Form: " + curr_row["Form"] + \
                       ", Question: " + curr_row["Question"] + "." + \
                       "}\n")
            file.write("\\label{fig:figure" + str(a_row) + "}\n")
            file.write("\\end{figure*}\n")
            file.write("\n")
        
            if discuss_or_not:
                file.write("\\noindent \\textbf{Discuss? {\color{red}{Yes}}}" + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Discuss? {\color{mgreen}{No}}}" + "\\\\\n")
            
            file.write("\\noindent Perry: ~~~~" + str(curr_row["PerryD"]) + "\\\\\n")
            file.write("Andrew: " + str(curr_row["AndrewD"]) + "\\\\\n")

            file.write("\n")
            file.write("\n")
            if is_there_disagreement:
                file.write("\\noindent \\textbf{Disagreement? {\color{red}{Yes}}}" + "\\\\\n")
                file.write("\\noindent Perry's Vote:~~~~ " + str(curr_row["PerryV"]) + "\\\\\n")
                file.write("Andrew's Vote: " + str(curr_row["AndrewV"]) + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Disagreement? {\color{mgreen}{No}}}" + "\\\\\n")
                file.write("\\noindent Perry's Vote:~~~~ " + curr_row['PerryV'] + "\\\\\n")
                file.write("Andrew's Vote: " + curr_row['AndrewV'] + "\\\\\n")

            file.write("\n")
            file.write("\n")
            file.write("\\noindent \\textbf{Comments}" + "\\\\\n")
            file.write("\\noindent Perry: ~~~~" + str(curr_row["PerryC"]) + "\\\\\n")
            file.write("Andrew: " + str(curr_row["AndrewC"]) + "\\\\\n")

            file.write("\\clearpage\n")

    file.write("disagreement count: " + str(disagreement_count) + "\\\\\n")
    file.write("\\end{document}\n")

In [24]:
all_responases.shape

(322, 20)

# Get rid of 144 repetions!

In [25]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
choices_set_1_xl = pd.ExcelFile(perry_dir + "set1_PerryandCo.xlsx")
choices_set_1_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names

chosen_fields_set1 = pd.DataFrame()
for a_choice_sheet in choices_set_1_sheet_names:    
    # read a damn sheet
    a_choice_sheet = choices_set_1_xl.parse(a_choice_sheet)
    chosen_fields_set1 = pd.concat([chosen_fields_set1, a_choice_sheet])

In [26]:
# all_responases = all_responases[~all_responases.ID.isin(list(chosen_fields_set1.ID.unique()))]

# out_name = perry_dir + "set2_all_responses_noRepetition.csv"
# all_responases.to_csv(out_name, index = False)

# Label for ML: change damn mustard to double-crop

In [ ]:
print (all_responases.shape)
print (all_responases.PerryV.unique())
print (all_responases.AndrewV.unique())

In [ ]:
A = all_responases.copy()
A = A[A.PerryV==A.AndrewV]
print (A.PerryV.unique())
print (A.AndrewV.unique())

In [27]:
double_place= ['Mustard Crop', 'Either double or mustard crop']

all_responases['PerryVCorrected']=all_responases['PerryV']
idx = all_responases[all_responases.PerryVCorrected.isin(double_place)].index
all_responases.loc[idx, "PerryVCorrected"] = 'Double Crop'
all_responases.head(2)

all_responases['AndrewVCorrected']=all_responases['AndrewV']
idx = all_responases[all_responases.AndrewVCorrected.isin(double_place)].index
all_responases.loc[idx, "AndrewVCorrected"] = 'Double Crop'
all_responases.head(2)
pre_meeting_agreements = all_responases[all_responases.PerryVCorrected==
                                                all_responases.AndrewVCorrected].copy()

# create Vote column in pre_meeting_agreements
pre_meeting_agreements['Vote'] = 1
pre_meeting_agreements.head(2)

# Change Vote to 2 for double cropped fields
double_index = pre_meeting_agreements[pre_meeting_agreements.PerryVCorrected=="Double Crop"].index
pre_meeting_agreements.loc[double_index, 'Vote']=2

pre_meeting_agreements = pre_meeting_agreements[["ID", "PerryV", "AndrewV", 
                                                 "PerryVCorrected", "AndrewVCorrected", 
                                                 "Vote"]]

out_folder = "/Users/hn/Documents/01_research_data/NASA/ML_data/"
out_name = out_folder + "set2_premeeting_consensus.csv"
pre_meeting_agreements.to_csv(out_name, index = False)

In [39]:
all_responases.head(2)

,ID,Set,Form,Question,PerryV,AndrewV,PerryD,AndrewD,PerryC,AndrewC,...,longitude,CropTyp,Irrigtn,DataSrc,Acres,ExctAcr,LstSrvD,county,PerryVCorrected,AndrewVCorrected
7,160820_WSDA_SF_2015,2,1,8,Single Crop,Single Crop,No,No,No,No,...,-118.646401,alfalfa seed,sprinkler,wsda,66,66.389879,2015/06/16 00:00:00,Walla Walla,Single Crop,Single Crop
8,161383_WSDA_SF_2015,2,1,9,Single Crop,Single Crop,No,No,No,No,...,-118.694748,alfalfa seed,sprinkler,wsda,77,76.906058,2015/06/04 00:00:00,Walla Walla,Single Crop,Single Crop


# DO NOT 
include the disagreement of type (double, mustard) in the PDF

In [49]:
os.chdir('/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/set_2_PDFs/')

disagreement_count = 0

with open("noRepetitions_set_2_3in1_Discussion_Comments_Disagreements.tex","w") as file:
    file.write("\\documentclass{article}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")
    file.write("\\definecolor{mgreen}{RGB}{25,147,100}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")
    

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

    file.write("\\title{\\bf Set 2 - Discussion, Comments, and Disgreements}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")


    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    # file.write("\\section{Discussion, Comments, and Disgreements}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in all_responases.index:
        curr_row = all_responases.loc[a_row]
        
        if len(np.unique(curr_row[["PerryVCorrected", "AndrewVCorrected"]].values))>1:
            is_there_disagreement=True
            disagreement_count+=1
        else:
            is_there_disagreement=False
        
        if str(curr_row["PerryD"])==str(curr_row["AndrewD"])=='No':
            discuss_or_not=False
        else:
            discuss_or_not=True
        
        if str(curr_row["PerryC"])==str(curr_row["AndrewC"])=='No':
            comment_or_not=False
        else:
            comment_or_not=True
        
        if is_there_disagreement or discuss_or_not or comment_or_not:
            TS_file_name = curr_row["NDVI_TS_Name"]
            file.write("\\begin{figure*}[ht]\n")
            file.write("\\centering\n")
            file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                         "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                             TS_file_name + "}\n")
            file.write("\\caption[]{" + \
                       "\\textbf{\\color{red}{" + curr_row["CropTyp"] + "}}" + \
                       ", ID: " + curr_row["ID"].replace("_", "\_") + \
                       ", Form: " + curr_row["Form"] + \
                       ", Question: " + curr_row["Question"] + "." + \
                       "}\n")
            file.write("\\label{fig:figure" + str(a_row) + "}\n")
            file.write("\\end{figure*}\n")
            file.write("\n")
        
            if discuss_or_not:
                file.write("\\noindent \\textbf{Discuss? {\color{red}{Yes}}}" + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Discuss? {\color{mgreen}{No}}}" + "\\\\\n")
            
            file.write("\\noindent Perry: ~~~~" + str(curr_row["PerryD"]) + "\\\\\n")
            file.write("Andrew: " + str(curr_row["AndrewD"]) + "\\\\\n")

            file.write("\n")
            file.write("\n")
            if is_there_disagreement:
                file.write("\\noindent \\textbf{Disagreement? {\color{red}{Yes}}}" + "\\\\\n")
                file.write("\\noindent Perry's Vote:~~~~ " + str(curr_row["PerryV"]) + "\\\\\n")
                file.write("Andrew's Vote: " + str(curr_row["AndrewV"]) + "\\\\\n")
            else:
                file.write("\\noindent \\textbf{Disagreement? {\color{mgreen}{No}}}" + "\\\\\n")
                file.write("\\noindent Perry's Vote:~~~~ " + curr_row['PerryV'] + "\\\\\n")
                file.write("Andrew's Vote: " + curr_row['AndrewV'] + "\\\\\n")

            file.write("\n")
            file.write("\n")
            file.write("\\noindent \\textbf{Comments}" + "\\\\\n")
            file.write("\\noindent Perry: ~~~~" + str(curr_row["PerryC"]) + "\\\\\n")
            file.write("Andrew: " + str(curr_row["AndrewC"]) + "\\\\\n")

            file.write("\\clearpage\n")

    file.write("disagreement count: " + str(disagreement_count) + "\\\\\n")
    file.write("\\end{document}\n")